In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim
import torch.utils.data.dataset as dataset
import numpy as np
from osgeo import gdal
import os
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

from dataset import *
from model import *

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
IMG_FOLDER = 'E:/xplore_data/data/'

## Extract features

In [4]:
# Load trained model
net = initialize_model2()
net.classifier = nn.Sequential(
    nn.Linear(net.n_features, 100),
    nn.Sigmoid(),
    nn.Linear(100, 20),
    nn.Sigmoid(),
    nn.Linear(20, 7)
)
n_features = 20

SAVED_MODEL_PATH = 'checkpoints/vgg11bn_4_e2e_all'
net.load_state_dict(torch.load(SAVED_MODEL_PATH))

# Freeze layers
for param in net.parameters():
    param.requires_grad = False
    
# We just want to apply the feature extractor for now
net.classifier[3] = nn.Identity()
net.classifier[4] = nn.Identity()

net.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(9, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [5]:
# Get dataset
DATA_FILE = 'E:/xplore_data/data/images.h5'
HEALTH_FILE = 'data/dhs_gps.csv'
dimages = TestDataset2(DATA_FILE, HEALTH_FILE)
dimagesloader = torch.utils.data.DataLoader(dimages, batch_size=64, shuffle=False, num_workers=0)

In [6]:
# Apply feature extractor to the dataset
n = len(dimages)
extracted_features = torch.zeros(n, n_features)
lights = torch.zeros(n, 3)
c_ids = np.zeros(n)
vac_rates = np.zeros((n,11))
i = 0
# Iterate over data.
net.to(device)
for x, lt, z in dimagesloader:
    x = x.to(device)
    j = i + x.shape[0]
    with torch.set_grad_enabled(False):
        outputs = net(x)
        extracted_features[i:j, :] = torch.squeeze(outputs).cpu()
        c_ids[i:j] = z[:, 0]
        vac_rates[i:j] = z[:, 14:25]
        lt = torch.reshape(lt, [-1, 333*333])
        lights[i:j, 0] = torch.sum(lt == 0, axis=1)
        lights[i:j, 1] = torch.sum(lt == 1, axis=1)
        lights[i:j, 2] = torch.sum(lt == 2, axis=1)
    i += x.shape[0]
extracted_features = extracted_features.numpy()
c_ids = c_ids.astype(np.int)

In [7]:
lights = lights.numpy()

In [7]:
torch.save(extracted_features, 'data/features_e2e.pt')

## Build dataset of built environment

In [8]:
extracted_features = torch.load('data/features_e2e.pt')

In [8]:
BUILT_FOLDER = 'E:/xplore_data/built/'
counts = np.zeros((889, 6), np.int)
for i, file in enumerate(os.listdir(BUILT_FOLDER)):
    img = load_file(BUILT_FOLDER, file)
    val, ct = torch.unique(img, return_counts=True)
    val = val.numpy().astype(np.int)-1
    ct = ct.numpy()
    counts[i, val] = ct

In [9]:
print(counts.shape)
print(extracted_features.shape)

(889, 6)
(889, 20)


In [10]:
# 0: water, 1: not built, 2-5: built from various times
built = counts[:, 2:].sum(axis=1)
water = counts[:, 0]
recent = counts[:, 2]
print(built.shape)

(889,)


In [11]:
from sklearn.linear_model import LinearRegression

def show_r2(x, y):
    reg = LinearRegression()
    x = x.reshape(889, -1)
    y = y.reshape(889, -1)
    reg.fit(x, y)
    print('%.3f' % reg.score(x, y))

## Check it out

In [12]:
# r2 between features and built 
from scipy.stats import pearsonr
for i in range(20):
    show_r2(built, extracted_features[:,i])

0.320
0.357
0.422
0.376
0.306
0.358
0.396
0.323
0.379
0.399
0.344
0.350
0.303
0.363
0.329
0.418
0.379
0.319
0.339
0.378


In [13]:
# Features and water
for i in range(20):
    show_r2(water, extracted_features[:,i])

0.000
0.001
0.000
0.000
0.000
0.001
0.000
0.000
0.000
0.000
0.000
0.000
0.000
0.000
0.000
0.001
0.000
0.001
0.002
0.001


In [21]:
# vaccs = [0, 1, 2, 3, 4, 5, 9]
vaccs = range(11)
stats = np.loadtxt('data/dhs_gps.csv', skiprows=1, delimiter=',')
vac_rates = stats[:, 14:]
lat = stats[:,1:3]
print(lat.shape)

(889, 2)


In [15]:
# Built and vaccination rate
for i in range(len(vaccs)):
    show_r2(built, vac_rates[:,vaccs[i]])

0.116
0.119
0.116
0.127
0.134
0.147
0.065
0.066
0.029
0.124
0.070


In [16]:
# What about water?
for i in range(len(vaccs)):
    show_r2(water, vac_rates[:,vaccs[i]])

0.000
0.000
0.000
0.000
0.000
0.000
0.000
0.001
0.000
0.000
0.000


In [22]:
# What about latitude and features?
for i in range(20):
    show_r2(lat, extracted_features[:,i])

0.563
0.530
0.431
0.486
0.610
0.533
0.509
0.537
0.495
0.452
0.534
0.516
0.551
0.527
0.569
0.502
0.420
0.604
0.593
0.538


In [25]:
# Latitude and vaccination rate?
for i in range(len(vaccs)):
    show_r2(lat, vac_rates[:,vaccs[i]])

0.519
0.400
0.512
0.497
0.456
0.367
0.244
0.209
0.054
0.463
0.220


In [26]:
# combination of latitude and built?
lat_built = np.concatenate((lat, built[:, None]), axis=1)

In [27]:
for i in range(20):
    show_r2(lat_built, extracted_features[:, i])

0.731
0.727
0.688
0.701
0.760
0.732
0.736
0.708
0.714
0.688
0.720
0.706
0.705
0.726
0.735
0.749
0.648
0.766
0.771
0.746


In [28]:
# Latitude and vaccination rate?
for i in range(len(vaccs)):
    show_r2(lat_built, vac_rates[:,vaccs[i]])

0.566
0.456
0.559
0.550
0.512
0.430
0.252
0.220
0.062
0.521
0.231


In [ ]:
resids = np.loadtxt('data/dhs_gps.csv', skiprows=1, delimiter=',')
resids[:, 14:] = 0
for i in range(len(vaccs)):
    y = vac_rates[:, vaccs[i]]
    reg = LinearRegression()
    reg.fit(lat_built, y)
    res = y - reg.predict(lat_built)
    resids[:, 14+vaccs[i]] = res

In [ ]:
np.savetxt('data/residuals.csv', resids, delimiter=',')

In [29]:
lights = lights[:, 1:]

In [30]:
for i in range(20):
    show_r2(lights, extracted_features[:,i])

0.446
0.469
0.475
0.452
0.441
0.468
0.478
0.450
0.466
0.467
0.430
0.448
0.418
0.455
0.450
0.505
0.452
0.447
0.483
0.478


In [31]:
for i in range(len(vaccs)):
    show_r2(lights, vac_rates[:,vaccs[i]])

0.296
0.272
0.290
0.302
0.304
0.293
0.152
0.149
0.045
0.288
0.118


In [32]:
lat_built_lights = np.concatenate((lat_built, lights), axis=1)
print(lat_built_lights.shape)

(889, 5)


In [33]:
for i in range(20):
    show_r2(lat_built_lights, extracted_features[:,i])

0.736
0.732
0.691
0.704
0.763
0.736
0.739
0.714
0.717
0.692
0.721
0.711
0.709
0.729
0.740
0.753
0.655
0.769
0.777
0.750


In [34]:
for i in range(len(vaccs)):
    show_r2(lat_built_lights, vac_rates[:,vaccs[i]])

0.575
0.471
0.567
0.560
0.525
0.447
0.269
0.240
0.077
0.531
0.237


In [ ]:
import lightgbm as lgb
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score as skl_r2
from sklearn.model_selection import KFold
mode = 'gbm'
for i in range(len(vaccs)):
    X = lat_built_lights
    y = vac_rates[:,vaccs[i]]
    n = X.shape[0]
    n_train = round(n*.8)
    shuffle = np.random.choice(range(n), size=n, replace=False)
    idx_train = shuffle[:n_train]
    idx_test = shuffle[n_train:]
        
    if mode == 'gbm':
        param_choices = utils.product_dict(
                objective=['regression'],
                num_leaves=(5, 7, 10),
                min_data_in_leaf=(5, 10, 15, 20)
                )
    elif mode == 'linear':
        param_choices = utils.product_dict(
        alpha=np.geomspace(.001, 100, num=6)
        )
    best_params = None
    best_mse = 999999999
    for params in param_choices:
        # Create CV folds
        n_fold = 10
        kf = KFold(n_splits = n_fold)
        pred = np.full_like(y[idx_train], np.nan)
        for train_folds, test_folds in kf.split(X[idx_train]):
            if mode == 'gbm':
                lgb_data = lgb.Dataset(data=X[train_folds], label=y[train_folds])
                gbm = lgb.train(params, lgb_data, 100)
                fold_pred = gbm.predict(X[test_folds])
            elif mode == 'linear':
                model = Ridge(**params, max_iter=1000)
                model.fit(X[train_folds], y[train_folds])
                fold_pred = model.predict(X[test_folds])
            pred[test_folds] = fold_pred
        avg_mse = np.power(y[idx_train]-pred, 2).mean()
        if avg_mse < best_mse:
            best_mse = avg_mse
            best_params = params
#            print('->', end='')
#        print(str(params) + ': %.3f' % r2)
    
    lgb_data = lgb.Dataset(data=X[idx_train], label=y[idx_train])
    best_gbm = lgb.train(best_params, lgb_data, 100)
    y_pred = best_gbm.predict(X)
    mse = [np.power(y[i]-y_pred[i], 2).mean() for i in (idx_train, idx_test)]
    r2 = [skl_r2(y[i], y_pred[i]) for i in (idx_train, idx_test)]
#    print('train/val/test r2 : %.3f / %.3f / %.3f' % tuple(r2))
#    print('train/val/test mse: %.3f / %.3f / %.3f' % tuple(mse))
    print(c.ljust(12) + ', %.3f, %.3f , %.3f, %.3f' 
          % (r2[0], mse[0], r2[1], mse[1]))